# Introduction
This will load the network's adjacency matrix A as a Scipy sparse matrix of shape _(N, N)_, the node features X of shape _(N, F)_, and the labels y of shape _(N, n_classes)_. The loader will also return some boolean masks to know which nodes belong to the training, validation and test sets

In [1]:
from spektral.datasets import citation
data = citation.load_data('cora')
A, X, y, train_mask, val_mask, test_mask = data
X = X.toarray()

N = A.shape[0] # N = 2708
F = X.shape[-1] #X.shape = 2708*1433  = 2708 nodes, 1433 features, F = 1433
n_classes = y.shape[-1] # n_classes = 7

Loading cora dataset
Pre-processing node features


In [2]:
print(n_classes)
print(y)
print(y.shape)

7
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2708, 7)


In [3]:
from spektral.layers import GraphConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout

Building the model is no different than building any Keras model, 
but we will need to provide multiple inputs to the GraphConv layers (namely A and X):

In [4]:

# Model definition
X_in = Input(shape=(F, ))  # This imply expected input will be batches of F-dimensional matrix (F=1433, input features)
A_in = Input((N, ), sparse=True)  # IThis imply expected input will be batches of N-dimensional matrix (N=2704, input adjacency), it is a sparse matrix.

graph_conv_1 = GraphConv(16, activation='relu')([X_in, A_in])
dropout = Dropout(0.5)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes, activation='softmax')([dropout, A_in])

# Build model
model = Model(inputs=[X_in, A_in], outputs=graph_conv_2)

In [5]:
X_in # <tf.Tensor 'input_5:0' shape=(None, 1433) dtype=float32>

A_in # <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1f922542160>

An important thing to notice at this point is how we defined the Input layers of our model. 

Because the "elements" of our dataset are the node themselves, we are telling Keras to consider each node as a separate sample, 
so that the "batch" axis is implicitly defined as None.
In other words, a sample of the node attributes will be _a row vector of shape (F, )_ and a sample of the adjacency matrix will be _one of its rows of shape (N, )_. (因為只需要睇該node的所有adjecency node)

Keep this detail in mind for later.

Before training the model, we have to pre-process the adjacency matrix to scale the weights of a node's connections according to its degree. 

In other words, the more a node is connected to others, the less relative importance those connections have. Most GNN layers available in Spektral require their own type of pre-processing in order to work correctly. You can find all necessary tools for pre-processing A in _spektral.utils_.

In [6]:
from spektral import utils
A = utils.localpooling_filter(A).astype('f4') #Thats all!! f4 = float, 4 byte long. see https://www.geeksforgeeks.org/data-type-object-dtype-numpy-python/ for details

In [7]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float32'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
graph_conv (GraphConv)          (None, 16)           22944       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 16)           0           graph_conv[0][0]             

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
# Train model
validation_data = ([X, A], y, val_mask)
model.fit([X, A],
          y,
          sample_weight=train_mask,
          epochs=100,
          batch_size=N, #batch size = no of nodes. Put all nodes into neural network at once.
          validation_data=validation_data,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[
              EarlyStopping(patience=10,  restore_best_weights=True)
          ])

# Evaluate model
print('Evaluating model.')
eval_results = model.evaluate([X, A],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)

eval_results

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 2708 samples, validate on 2708 samples
Epoch 1/100
2708/2708 [==============================] - 2s 780us/sample - loss: 0.1006 - acc: 0.0857 - val_loss: 0.3592 - val_acc: 0.1640
Epoch 2/100
2708/2708 [==============================] - 0s 66us/sample - loss: 0.1006 - acc: 0.0714 - val_loss: 0.3592 - val_acc: 0.1920
Epoch 3/100
2708/2708 [==============================] - 0s 78us/sample - loss: 0.1005 - acc: 0.2000 - val_loss: 0.3591 - val_acc: 0.2140
Epoch 4/100
2708/2708 [==============================] - 0s 66us/sample - loss: 0.1005 - acc: 0.2214 - val_loss: 0.3590 - val_acc: 0.2520
Epoch 5/100
2708/2708 [==============================] - 0s 63us/sample - loss: 0.1004 - acc: 0.2786 - val_loss: 0.3589 - val_acc: 0.3060
Epoch 6/100
2708/2708 [==============================] - 0s 70us/sample - loss: 0.1004 - acc: 0.3429 - val_loss: 0.3589 - val_acc: 0.3320
Epoch 7/100
2708/2708 [==============================] - 0s 66us/sample -

[0.6860324144363403, 0.775]

In [10]:
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

Done.
Test loss: 0.6860324144363403
Test accuracy: 0.7749999761581421


In [14]:
import numpy as np

# one hot encoder to int
ground_truth = [np.where(r==1)[0][0] for r in y]

y_result = model.predict([X,A], batch_size=N)
y_group = []
for index, item in enumerate(y_result):
    y_group.append(np.argmax(y_result[index]))


from sklearn import metrics
print(metrics.adjusted_rand_score(ground_truth, y_group))
print(metrics.adjusted_mutual_info_score(ground_truth, y_group))
print(metrics.accuracy_score(ground_truth, y_group))
print(ground_truth)
print(y_group)

0.5727096486108213
0.5585460768754362
0.7843426883308715
[3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6, 3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5, 3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2, 2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0, 6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5, 5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 3, 0, 3, 6, 6, 0, 3, 4, 0, 3, 4, 4, 1, 2, 2, 2, 3, 3, 3, 3, 0, 4, 5, 0, 3, 4, 3, 3, 3, 2, 3, 3, 2, 2, 6, 1, 4, 3, 3, 3, 6, 3, 3, 3, 3, 0, 4, 2, 2, 6, 5, 3, 5, 4, 0, 4, 3, 4, 4, 3, 3, 2, 4, 0, 3, 2, 3, 3, 4, 4, 0, 3, 6, 0, 3, 3, 4, 3, 3, 5, 2, 3, 2, 4, 1, 3, 2, 2, 3, 3, 3, 3, 5, 1, 3, 1, 3, 5, 0, 3, 5, 0, 4, 2, 4, 2, 4, 4, 5, 4, 3, 5, 3, 3, 4, 3, 0, 4, 5, 0, 3, 6, 2, 5, 5, 5, 3, 2, 3, 0, 4, 5, 3, 0, 4, 0, 3, 3, 0, 0, 3, 5, 4, 4, 3, 4, 3, 3, 2, 2, 3, 0, 3, 1, 3, 2, 3, 3, 4, 5, 2, 1, 1, 0, 0, 1, 6, 1, 3, 3, 3, 2, 3, 